<a href="https://colab.research.google.com/github/hargurjeet/Adhoc-Activities/blob/main/Neural_Nets_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os, imageio, zipfile
import imageio.v2 as imageio
# from scipy.misc import imread, imresize
import datetime
import os
from skimage import io
from skimage.transform import resize
import numpy as np

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# # Specify the path to the zip file
# zip_file_path = r'/content/drive/MyDrive/Upgrad/Gesture Recognition/Project_data.zip'

# # Create a ZipFile object
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     # Extract all the contents of the zip file to a folder
#     zip_ref.extractall('/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data')

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/val.csv').readlines())
batch_size = 10

In [6]:
import os

# specify the directory path
path = "/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train"

# get all the folder names in the directory
folder_names = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]

# print the folder names
print(folder_names)

['WIN_20180907_15_35_09_Pro_Right Swipe_new', 'WIN_20180907_15_38_17_Pro_Left Swipe_new_Left Swipe_new', 'WIN_20180907_15_38_24_Pro_Right Swipe_new', 'WIN_20180907_15_38_35_Pro_Thumbs Down_new', 'WIN_20180907_15_39_51_Pro_Stop Gesture_new', 'WIN_20180907_15_39_54_Pro_Thumbs Up_new', 'WIN_20180907_15_40_26_Pro_Thumbs Down_new', 'WIN_20180907_15_40_55_Pro_Thumbs Up_new', 'WIN_20180907_15_41_54_Pro_Right Swipe_new', 'WIN_20180907_15_43_00_Pro_Left Swipe_new_Left Swipe_new', 'WIN_20180907_15_43_28_Pro_Right Swipe_new', 'WIN_20180907_15_43_40_Pro_Stop Gesture_new', 'WIN_20180907_15_43_58_Pro_Left Swipe_new_Left Swipe_new', 'WIN_20180907_15_44_13_Pro_Stop Gesture_new', 'WIN_20180907_15_44_32_Pro_Thumbs Down_new', 'WIN_20180907_15_44_36_Pro_Right Swipe_new', 'WIN_20180907_15_44_55_Pro_Stop Gesture_new', 'WIN_20180907_15_45_30_Pro_Thumbs Up_new', 'WIN_20180907_15_45_40_Pro_Left Swipe_new_Left Swipe_new', 'WIN_20180907_15_45_56_Pro_Thumbs Down_new', 'WIN_20180907_15_46_25_Pro_Stop Gesture_new',

In [7]:
import cv2

# Load the image using cv2.imread()
img = cv2.imread('/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png')

# Get the dimensions of the image
height, width, channels = img.shape

# Print the dimensions
print("Image dimensions: {} x {} x {}".format(width, height, channels))

Image dimensions: 360 x 360 x 3


In [8]:
# Total images within the dataset
# import glob

# path = "/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train/*/*.png"  # replace with the path to your directory
# count = len(glob.glob(path))

# print("Total number of images:", count)

In [9]:
# %%time

# ## Code to indentify the different dimension present in the datset
# from PIL import Image
# import os
# import glob

# for folder in folder_names:
#   counter = []
#   directory = "/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train/" + folder
  
  # count = len(glob.glob(directory))
  # counter.append(count)
  # print(directory)
  # for filename in os.listdir(directory):
  #     if filename.endswith(".png"):
  #         file_path = os.path.join(directory, filename)
  #         with Image.open(file_path) as img:
  #             width, height = img.size
  #             print(f"{directory}: {width} x {height}")
              # image_dim.append(img.size)

# print(counter)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [78]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(3)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 31 # calculate the number of batches, (19890/(32*20)= 31 )
        remaining_datapoints = len(folder_list) % batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,20,160,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # Set the new width and height
                    new_width = 120
                    new_height = 160

                    # Resize the image
                    resized_img = resize(image, (new_height, new_width))
                    
                    channel = resized_img[:, :, 0]
                    channel_normalized = (channel - channel.min()) / (channel.max() - channel.min())
                    batch_data[folder,idx,:,:,0] = channel_normalized

                    channel = resized_img[:, :, 1]
                    channel_normalized = (channel - channel.min()) / (channel.max() - channel.min())
                    batch_data[folder,idx,:,:,1] = channel_normalized

                    channel = resized_img[:, :, 2]
                    channel_normalized = (channel - channel.min()) / (channel.max() - channel.min())
                    batch_data[folder,idx,:,:,2] = channel_normalized
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches

        if remaining_datapoints > 0:
            batch_data = np.zeros((remaining_datapoints,20,160,120,3))
            batch_labels = np.zeros((remaining_datapoints,5))
            for folder in range(remaining_datapoints):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imageio.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    new_width = 120
                    new_height = 160

                    resized_img = resize(image, (new_height, new_width))
                    
                    channel = resized_img[:, :, 0]
                    channel_normalized = (channel - channel.min()) / (channel.max())
                    batch_data[folder,idx,:,:,0] = channel_normalized

                    channel = resized_img[:, :, 1]
                    channel_normalized = (channel - channel.min()) / (channel.max() - channel.min())
                    batch_data[folder,idx,:,:,1] = channel_normalized

                    channel = resized_img[:, :, 2]
                    channel_normalized = (channel - channel.min()) / (channel.max() - channel.min())
                    batch_data[folder,idx,:,:,2] = channel_normalized


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [79]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/train'
val_path = '/content/drive/MyDrive/Upgrad/Gesture Recognition/Unzipped data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 3 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 3


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [80]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()

model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', input_shape=(20,160,120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [81]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 18, 158, 118, 64)  5248      
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 9, 79, 59, 64)    0         
 g3D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 9, 79, 59, 64)     0         
                                                                 
 conv3d_13 (Conv3D)          (None, 7, 77, 57, 32)     55328     
                                                                 
 max_pooling3d_13 (MaxPoolin  (None, 3, 38, 28, 32)    0         
 g3D)                                                            
                                                                 
 dropout_19 (Dropout)        (None, 3, 38, 28, 32)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [82]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [98]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss: f}-{val_categorical_accuracy:.5f}.h5'

# filepath= 'model_init_2023_05_13-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss: f}-{val_categorical_accuracy:.5f}.h5'

# filepath= 'model_init_2023_05_13-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = 'model_init_2023_05_13-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [99]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
# else:
#     steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
# else:
#     validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [100]:
%%time
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data= val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/3


KeyError: ignored